# Workflow for Diffuse/Direct - Mullion - 11SEP20

## Instructions

This workflow is designed such that you should only need to edit information in the first cell below.

The workflow assumes that diffuse/direct measurements have been taken with either 6 or 8 spectra for each measurement. If any other number of spectra is found, all of them will be ignored. It is assumed that with 6 spectra, the oder of measurement is Direct, Helper, Shade, with two spectra for each part. If there are 8 spectra, it is assumed that the last two are Helper.
    
### Input and Output directories
Specify input directory as 'indir' and output directory as 'outdir'. Remember to include trailing slash (/) in path name.

### Field data information
The "field_data" list should contain basic information on the location and date of the diffuse/direct measurements. The first entry is the field site three-letter-acronym (eg "MUL" for Mullion), the second entry is the date in DDMMMYY format.

### Prefix list
'PreList' contains substrings that are searched for in order to identify diffuse/direct spectra. If your spectral
files contain any of these, then the workflow will pick them up. Other substrings can be added to this list.

### Suffix
Since it is possible that not all ASD spectra files have the same filename suffix, this is manually entered as "asd.irr.txt". Pay close attention to the full stops in this string. ie. ".asd.irr.txt" will not work.


Once you have all of the fields in the first cell correctly filled out, you can click on "Kernel" -> "Restart & Run All". Scroll down to the bottom to see results.

In [ ]:
#
# Start a clock to time the workflow
#
import time
start_time = time.time()

#
# Specify input and output directories below
#
#indir = '/g/data/up71/projects/CalVal_Phase2/SR3500/11SEP20/RAW_DATA/ASD_DATA/direct_diffuse/ALL/'
indir = '/g/data/u46/users/gtb547/cal_val/in_spectra/Mullion/Mull_13_2020_9_nov_s2b/atmosphere/direct_diffuse/end/'
outdir = '/g/data/up71/projects/CalVal_Phase2/SR3500/11SEP20/PNGS/TMP/'

#
# Specify an input image to overplot on spectra
#
import matplotlib.pyplot as plt
from matplotlib.cbook import get_sample_data
im = plt.imread(get_sample_data('/g/data/u46/users/gtb547/cal_val/in_spectra/Mullion/Mull_12_2020_11_sept_s2b/photos/20200911_102803.jpg'))

#
# Fill out field_data below. It is in the format: 'Field Site Short Name', 'Date'.
#
field_data = ['MUL', '11SEP20']

#
# List of prefixes to pattern match DD files on
#
PreList = ['dd', 'diffdir', 'dif_dir', 'irrads', 'direct_diffuse']

#
# Assumed suffix for spectral files is defined below
#
#suffix = 'asd.irr.txt'
suffix = '.txt'


## Import plotting packages and set up png outputs

In [ ]:
import numpy as np
import pandas as pd
import matplotlib

import datacube
import sys, os, shutil, glob, subprocess

from datetime import datetime, timedelta


#
# Use notebook format
# Set default font size for all plots
# Set saved figure dpi to high quality
#
%matplotlib inline
matplotlib.rcParams.update({'font.size': 12})
matplotlib.rcParams['savefig.dpi'] = 300

#
# Remove old files in output directory and create a new one
#
directory = os.path.dirname(outdir)
if os.path.exists(directory):
    shutil.rmtree(directory)
os.makedirs(directory)

## Set up colours used for plotting spectra

In [ ]:

#
# Colours used for plotting multi-coloured Lines
#
colpac=[['#770000', '#FF0000'],
        ['#007700', '#00FF00'],
        ['#000077', '#0000FF'],
        ['#777700', '#FFFF00'],
        ['#007777', '#00FFFF'],
        ['#770077', '#FF00FF']]


#
# Start Figure numbering at 1
#
fignum = 1

### The following cell was originally "LoadData.py", but brought internally so the notebook doesn't depend on anything externally.

In [ ]:
###############################################################################
#                                                                             #
# Action functions are defined to retrieve specific parts of the header for   #
# each spectrum. These functions are used in extract_metadata.                #
#                                                                             #
###############################################################################

# Instrument Number
def action1(l):
    return l[27:34]

# Datetime of spectrum
def action2(l):
    return l[16:38]

# SWIR1 gain
def action3(l):
    return l[15:33]

# SWIR2 gain
def action4(l):
    return l[15:33]

# GPS Latitude in decimal degrees
def action5(l):
    try:
        lat = float(l[17:20])-float(l[20:27])/60
    except ValueError:
        lat = 0
    print('Lat = ', lat)
    return lat

# GPS Longitude in decimal degrees
def action6(l):
    try:
        lon = float(l[19:22])+float(l[22:30])/60
    except ValueError:
        lon = 0
    print('Lon = ', lon)
    return lon

#
# Based on action functions defined above, extract header metadata from
# a file.
#
def extract_metadata(filename):
    strings = {
        'instrument number': action1,
        'Spectrum saved': action2,
        'GPS-Latitude': action5,
        'GPS-Longitude': action6
    }

    with open(filename) as file:
        list_of_actions = []
        for line in file:
            for search, action in strings.items():
                if search in line:
                    list_of_actions.append(action(line))
        return list_of_actions

#
### Extract spectrum and header information from a spectrum file. 
### Create a Pandas dataframe with the result.
#
def load_spectrum_to_df(infile, i):
    
    p1 = subprocess.Popen(["grep", "-an", "^Wavelength", infile], stdout=subprocess.PIPE)
    p2 = subprocess.Popen(["cut", "-d:", "-f", "1"], stdin=p1.stdout, stdout=subprocess.PIPE)
    p1.stdout.close()  # Allow p1 to receive a SIGPIPE if p2 exits.
    fdl,err = p2.communicate()
    firstDataLine = int(fdl)-1

    inst, date_str, lat, lon = extract_metadata(infile)

    date_saved = datetime.strptime(date_str, '%m/%d/%Y at %H:%M:%S')
    
    df = pd.read_csv(infile, skiprows=firstDataLine, delim_whitespace=True)
    filename = df.columns[1]
    df.rename({filename: 'radiance'}, axis=1, inplace=True)
    df['filename'] = filename
    df['date_saved'] = date_saved
    df['Latitude'] = lat
    df['Longitude'] = lon
    df['Type'] = i
    try:
        df['Spec_number'] = int(filename[-10:-8])
    except ValueError:
        df['Spec_number'] = int(filename[-6:-4])
    df['Inst_number'] = inst
    return df

#
### Loop through all spectrum files in "indir" and combine the resulting dataframes.
#
# For each 'line*' directory in 'indir', iterate through each file
# ending with 'suffix' and run 'load_spectrum_to_df'. Finally,
# return a concatenated dataframe made up of all the individual
# dataframes.
#
def load_from_dir(indir, prefix, suffix):
    all_dfs = []
    
    #
    # Initalise 'spectra' list and fill with files that end in 'suffix'
    #
    spectra = []
    for root, dirs, files in sorted(os.walk(indir)):
        for file in files:
            if file.startswith(prefix) and file.endswith(suffix):
                spectra.append(file)
    spectra = sorted(spectra)

    for name in spectra:
    
        infile = indir + name
    
        df = load_spectrum_to_df(infile, prefix)
        all_dfs.append(df)

    return pd.concat(all_dfs)

## Search for, and identify, unique prefixes for multiple measurements made on the same day

In [ ]:
def unique_prefixes(indir, PreList, suffix):
    spectra = []
    for root, dirs, files in sorted(os.walk(indir)):
        for file in files:
            if any(ext in file for ext in PreList) and file.endswith(suffix):
                spectra.append(file)

    prefixes = []
    for i in set([x[:-17] for x in spectra]):
        prefixes.append(i)
        
    for pfx in prefixes:
        spec2 = []
        for file in spectra:
            if file.startswith(pfx):
                spec2.append(file)

        if len(spec2) != 6:
            if len(spec2) != 8:
                print('Removed spectra with sub-string '+pfx+': there are '+str(len(spec2))+' spectra and NOT 6 or 8, as expected.')
                prefixes.remove(pfx)
        else:
            print('Added '+str(len(spec2))+' spectra with prefix '+pfx)

    return prefixes

prefs = unique_prefixes(indir, PreList, suffix)

## Make DataFrame for all measurements

In [ ]:
def make_DDframe(indir, prefs, suffix):
    
    alldfs= []
    
    for i in prefs:
        df = load_from_dir(indir, i, suffix)
        alldfs.append(df)

    return pd.concat(alldfs)

alldfs = make_DDframe(indir, prefs, suffix)

## Create Diffuse/Direct DataFrame for one measurement

In [ ]:
def DiffuseDirect(alldata):
    
    intdata = alldata.copy()
    intdata.set_index('Wavelength', inplace=True)

    df=pd.DataFrame()
    for i in intdata.Spec_number.unique():
        df[str(i)] = intdata[intdata.Spec_number==i].radiance

    Direct = df[['0','1']].mean(axis=1)
    try:
        Helper = df[['2','3', '6', '7']].mean(axis=1)
    except KeyError:
        Helper = df[['2','3']].mean(axis=1)

    Shade = df[['4','5']].mean(axis=1)
    Diffuse = Direct - (Helper - Shade)
    outdf = pd.concat([Diffuse, Direct], axis=1)
    outdf.columns=['Diffuse', 'Direct']

    return outdf

## Plot diffuse and direct spectra for each measurement

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(9,5))
plt.tight_layout(pad=3.5, w_pad=1.0, h_pad=1.0)

leglab = []

for idx, i in enumerate(prefs):
    SingleDF = alldfs[alldfs.Type==i]
    outdf = DiffuseDirect(SingleDF)
    outdf.plot(ax=axes, color=colpac[idx])
    ts = pd.to_datetime(str(alldfs[alldfs.Type==i].date_saved.unique()[0])) 
    hour = int(ts.strftime('%H'))
    mins = ts.strftime('%M')
    time_s = str((hour+10)%12)+':'+mins+' AEST'
    leglab.append('Diffuse '+time_s)
    leglab.append('Direct '+time_s)

axes.legend(labels=leglab)
axes.set_ylim(-0.02, 2.0)
axes.set_xlim(330,2400)
axes.set_xlabel('Wavelength (nm)')
axes.set_ylabel('Radiance (W m$^{-2}$ nm$^{-1}$ sr$^{-1}$)')
plt.title(field_data[0]+' '+field_data[1])

newax = fig.add_axes([0.6,0.4,0.3,0.3], anchor='NE', zorder=1)
newax.imshow(im)
newax.axis('off')

plt.savefig(outdir+'dd.png')

## How long did this notebook take to run?

In [ ]:
import datetime
print("This Notebook took ", str(datetime.timedelta(seconds=int((time.time() - start_time)))), "(h:m:s) to run")